In [1]:
import ps
import ps_utils
import numpy as np
import pandas as pd
import datetime
ps_utils.switch_to_prod_pvlss()
#ps_utils.switch_to_psi()
import matplotlib.pyplot as plt
#ps_utils.switch_to_psi()
dupire='DupireLocalVolatilityModel'
no_skew_model = 'HybridLGM1FDupireLocalVolatilityModel'
skew_model = 'HybridCheyette1FactorLocalVolatilityModel'
ps_utils.global_algo_id()

[ERROR] 2023-07-03 11:39:32,920 - You do not have xbbg installed, some functionality will not be available
[INFO] 2023-07-03 11:39:33,337 - Nothing to do: current environment is already prod_pvlss


'6c21bbf749e70ca1d53f365a33d8b184691c53c2'

In [2]:
ps_utils.__file__

'C:\\Users\\19865272\\strat\\src\\ps_utils\\__init__.py'

In [3]:
ps_utils.get_env_url('psi')

'http://tkles-dealm0004.vm.esrt.cloud.sbrf.ru'

In [4]:
def get_bcva_df(a):
    print('PV=',a['ProductValue'])
    cva_vals=[]
    cva_mas1=[]
    cva_mas2=[]
    dva_vals=[]
    dva_mas1=[]
    dva_mas2=[]
    for el in a:
        if el.startswith('CVA'):
            if type(a[el])!=ps.message.Dict:
                cva_mas1.append(el[1:])
                cva_mas2.append(el[1:])
                cva_vals.append(a[el])
            if type(a[el])==ps.message.Dict:
                for k in a[el]:
                    cva_mas1.append(el[4:])
                    cva_mas2.append(k)
                    if (el.endswith('Exposure')):
                        cva_vals.append(a[el][k])
                    elif (el.endswith('Delta') or el.endswith('Rho') or el.endswith('RatesVega')):
                        cva_vals.append(a[el][k]/10000)
                    elif (el.endswith('ExchangeVega')):
                        cva_vals.append(a[el][k]/100)
                    else:
                        print('Unexpected item:',el)
        elif el.startswith('DVA'):
            if type(a[el])!=ps.message.Dict:
                dva_mas1.append(el[1:])
                dva_mas2.append(el[1:])
                dva_vals.append(a[el])
            if type(a[el])==ps.message.Dict:
                for k in a[el]:
                    dva_mas1.append(el[4:])
                    dva_mas2.append(k)
                    if (el.endswith('Exposure')):
                        dva_vals.append(a[el][k])
                    elif (el.endswith('Delta') or el.endswith('Rho') or el.endswith('RatesVega')):
                        dva_vals.append(a[el][k]/10000)
                    elif (el.endswith('ExchangeVega')):
                        dva_vals.append(a[el][k]/100)
        else:
            pass
    cva_arrays = [np.array(cva_mas1),np.array(cva_mas2)]
    cva_df=pd.DataFrame(cva_vals, index=cva_arrays)
    dva_arrays = [np.array(dva_mas1),np.array(dva_mas2)]
    dva_df=pd.DataFrame(dva_vals, index=dva_arrays)
    #dfotv=pd.concat([cva_df, dva_df], axis=1)
    #dfotv.fillna(0, inplace=True)
    vals=np.zeros(len(cva_df.values)*3).reshape(-1,3)
    vals[:,0]=cva_df.values.flatten()
    vals[:,1]=dva_df.values.flatten()
    vals[:,2]=(cva_df.values+dva_df.values).flatten()
    #(cva_df.values+dva_df.values).shape
    #dva_df=pd.DataFrame(dva_vals, index=dva_arrays)
    return pd.DataFrame(vals, index=cva_df.index)


In [66]:
#mas=[]
r=ps.get('e32436c7fea7522996787a6f66670afb91147792')
for el in r.Result:
    if(ps.get(el._nonce)['_Counterparty'] in ['SIBRH']):
        print(ps.get(el._nonce)['_Counterparty'],el._id)
        #mas.append(el._id)
#mas

SIBRH a1e5e9b110fe3b524183ead7001a93831452b83d


In [68]:
-4.153685129984903e+06+3.9870177149032303e+06

-166667.41508167284

In [ ]:
wanted=['DELMZ','VITMG', 'SBGGG']
mas=[]
r=ps.get('224c95e27ce3e559640f7f73ff221f6bd379fda9')
for el in r.Result:
    if(ps.get(el._nonce)['_Counterparty'] in wanted ):
        mas.append(ps.get(el._nonce)._id)
r=ps.get(ps.put(ps.new_cm(mas)))
for el in r:
    el.Queries=['ProductValue','BCVA']
z=ps_utils.compute(ps.put(r))
for i,el in enumerate(z):
    print(el['ProductValue'],el['BCVA'],ps.get(r[i]._id)['_Counterparty'])

In [42]:
#mas=[]
r=ps.get('7ac40ba63ecb042f46d138d3ad8133668e1b20a1')
for el in r.Result:
    try:
        s=el.StatusText
    except:
        s='None'
    if (s!='done'):
        print(ps.get(el._nonce)['_Counterparty'], ps.get(el._nonce).Product.CounterpartyCreditRating, el._id,s[0:65])
        #mas.append(ps.get(el._nonce)._id)
#mas

PTROL 17 2ca5f7284132b179895b4e54a5bbccb6563d5cca Unable to find index AI95K5_YAI_SPX_AVG in the static data set. w
FSKSS 8 21b104ff94217758fc2a0c39c931b72dcd8c2af6 Unable to find index RU_CPI_2000 in the static data set. while co
MBDDR 12 72110ae3b9d0e6a35985661c4d68a7f8b5a7e2df Unable to find index INRRUB in the static data set. while constru
KCVET 14 6fd5d420283b0b7f89a050b121643d30b27b8776 Failed to locate IRVol[XPT_USD] in market while constructing DT_H
SOLFE 15 4dcf4f280adc9c938ca0239f3eb3bce81cbddd8c Failed to locate Correlation[USDRUB&XAGRUB] in market while const
MTSSS 10 49027e68c906174fc0ac26e93b10c232a2dfd857 Failed to locate fixing for USD_SOFR for date 2016-12-30T11:00:00
AVGUT 8 575c005bc7d9e44f7a4702443c316bdb1250661b Failed to locate BYN in CalibrationUnderlyings while constructing
MBMMR 10 6aa254331c91b4a898b330a7758174b2373d78bd Unable to find index INRRUB in the static data set. while constru
DIEKS 8 556108e4c8e72bcb33e5340f6175a0a0fc260474 Failed to locate BYN in C

In [39]:
import ps, ps_utils
_from = 'dev'; _to = 'prod_pvlss'
#_from = 'prod_pvlss'; _to = 'dev'
_id = '0e85ca53c8f6dc9d873041529a4951be2c1f0a06'

ps_utils.upload_message(_id, _from, _to)

ps_utils.switch_to_env(_to)
print(f"check result")
n, not_found = ps_utils.chk_msg_tree(ps.get(_id))
print(f"\ndone: {n}, not found: {not_found}\n")
if not_found:
    print(f"copy missing messages")
    for m_id in not_found:
        ps_utils.upload_message(m_id, _from, _to)
    print(f"check result")
    n, not_found = ps_utils.chk_msg_tree(ps.get(_id))
    print(f"\ndone: {n}, not found: {not_found}\n")


[INFO] 2023-07-03 14:15:26,463 - Uploaded message 0e85ca53c8f6dc9d873041529a4951be2c1f0a06 from dev to prod_pvlss
[INFO] 2023-07-03 14:15:26,466 - Nothing to do: current environment is already prod_pvlss


check result

done: 2566, not found: []



# 3 июля 2023 понедельник

# Тема 18 рейтинг  FSKCD030723 Package booked (Package number: "44362367").

In [12]:
#r=ps.get('19ca44cac520b7734029955501f9569d290a3bf9')

           
#Greeks new deal
r=ps_utils.pricing.XVA(ps.get('264e7d64a9527488e72657fbf573f8fddb887f7e')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])


#r.Model.MarketDataSet='18ff5367c65fcd43972a067c7888ba9c23e55415'

#r.Product.NettingSets[0].MarginSets[0].Products=['00400afe5a72000da2fd9dd36d5d55808b233f2f']
#r.Model.MarketDataSet='b8207adfdcb659a431753bd7a4e8ff1a77f6befc'
#r.Model.MarketDataSet.Spots['CNHRUB']=11.31

#r.set_correlation('EURRUB','CNHRUB',0.8)

# r.Product.NettingSets[0].MarginSets[0].Products.pop(0)
# r.Product.NettingSets[0].MarginSets[0].Products.pop(1)

# r.set_correlation('USDRUB','XAURUB',0.6)
# r.set_correlation('USD_LIBOR_3M','XAURUB',0.0)
# r.set_correlation('USDRUB','XAU_USD',0.0)
# r.set_correlation('USD_LIBOR_3M','XAU_USD',0.0)

#r.Model.MarketDataSet.Correlations['CNHRUB&RUB_KEY_RATE']=0.3

# # # r.set_mean_reversion('RUB_KEY_RATE',0.05)
# # # r.set_correlation('USDRUB','CNHRUB',0.7)
#r.Model.TypeName=skew_model

# # #r.set_nb_paths(100000)

#r.set_correlation('USDRUB','CNHRUB',0.8)

# #r.set_model_parameter('AmericanMonteCarloNumberOfPaths',10240)

# r.Product.pop('CounterpartyCreditRating')
# r.Product.CounterpartyCreditCurveIdentifier='RUB_CHMF_OFZ_SNR_CR'

# new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.8, 'RUB_CHMF_OFZ_SNR_CR': 1.49}
# for cr_curve, cr_value in new_credit_values.items():
#     r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
#                                                                        len(r.Model.MarketDataSet.CreditCurves[
#                                                                                cr_curve].RatesSchedule.Rates)


# r.Model.MarketDataSet.Correlations['CNHRUB&RUB_KEY_RATE']=0.3


# r.Model.MarketDataSet.RatesCurvesBundles['RUB_KEY_RATE'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['EUR_XCCY'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['CNH_FX'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['RUB_SOFR'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['USD_SOFR'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['RUB_RUONIA_OIS'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['EUR_EURIBOR_6M'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['EUR_RUB'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['USD_SOFR'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['USD_LIBOR_3M'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['USD_XCCY'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['XAU_USD'].InterpolationMethod='Linear'

# r.Product.pop('CounterpartyCreditRating')
# r.Product.CounterpartyCreditCurveIdentifier='RUB_CHMF_OFZ_SNR_CR'

# new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.8, 'RUB_CHMF_OFZ_SNR_CR': 6.00}
# for cr_curve, cr_value in new_credit_values.items():
#     r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
#                                                                        len(r.Model.MarketDataSet.CreditCurves[
#                                                                                cr_curve].RatesSchedule.Rates)

a=r.price(algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
get_bcva_df(a)

[INFO] 2023-07-03 12:23:03,014 - Compute:a8e6ea0bf6933779315b9b6e8a5f8bff60af5d3f->e7319e6c815ac3f3f7a3ddeb74cf827e14d05f4d


Dict([('BCVA', -178756.14266496888),
      ('LegBCVA', [-144547.01346153158, -34223.02625713992]),
      ('Legs',
       [1b624da946aa683e9da3ac6c6f31f27b2662c28c,
        6e8cf02cf569261d645f2b8f93c9a0649b7ca9c1]),
      ('CVA', -181732.3342124842),
      ('DVA', 2976.1915475152973),
      ('LegPresentValues', [1243778.5097299085, 273427.0680831617]),
      ('ProductValue', 1517205.57781307),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 6548.875268071165),
      ('CashToSettleToday', 0.0)])

In [7]:
z=r.run_xva_breakdown(show=True)

[INFO] 2023-07-03 11:59:44,726 - Compute:ee7198d9c5fa3334985eb517d58b72c8a3043e18->746f167dc5ac2b85a4eac47a884f3287c9ae7e6c


In [8]:
#44362368
(104199099-21150)/r.Model.MarketDataSet.Spots['USDRUB']

1170292.2024854315

In [9]:
#44362369
(22643038-4596)/r.Model.MarketDataSet.Spots['USDRUB']

254310.9401109317

In [10]:
#r=ps.get('19ca44cac520b7734029955501f9569d290a3bf9')

           
#Greeks new deal
r=ps_utils.pricing.XVA(ps.get('264e7d64a9527488e72657fbf573f8fddb887f7e')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])



a=r.price(algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
get_bcva_df(a)

[INFO] 2023-07-03 12:06:28,875 - Compute:b00b23afb7450268052f09f9f6fda076beffe833->387d2fe4e88285e84610030e6f5a93d96b1ee09e


PV= 1514817.5091396978


0        1           2
VA                        VA                  -181,676.31 2,975.99 -178,700.32
ParallelInterestRatesVega RUB_KEY_RATE            -213.16    36.64     -176.52
ParallelCreditDelta       RUB_MINFIN_OFZ          -304.64     0.00     -304.64
                          RUB_SBER_OFZ_SNR_CR        0.00    29.08       29.08
ParallelRho               RUB_KEY_RATE            -850.34   -20.34     -870.68
                          RUB_RUONIA_OIS            41.47    -0.62       40.84
                          RUB_SOFR                   0.00     0.00        0.00
                          USD_SOFR                  -0.00    -0.00       -0.00
CurrencyExposure          RUB                 -181,676.31 2,975.99 -178,700.32
                          USD                        0.00    -0.00        0.00

In [13]:
870*10

8700

In [15]:
145000*1.02

147900.0

In [17]:
35000*1.02

35700.0

In [18]:
147000+35700

182700

In [25]:
#r=ps.get('19ca44cac520b7734029955501f9569d290a3bf9')

           
#Greeks new deal
r=ps_utils.pricing.XVA(ps.get('264e7d64a9527488e72657fbf573f8fddb887f7e')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
#r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])



r.Product.CounterpartyCreditRating=8


r.price(algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-07-03 13:19:27,581 - Compute:62eac97f653f07e2d4697585ee803e234580bcfd->d23ed5d2451bab64e894a61d280957e1f33a3645


Dict([('BCVA', -32734.49485981069),
      ('LegBCVA', [-26534.321236137173, -6200.173623673537]),
      ('Legs',
       [1b624da946aa683e9da3ac6c6f31f27b2662c28c,
        6e8cf02cf569261d645f2b8f93c9a0649b7ca9c1]),
      ('CVA', -35710.686407325986),
      ('DVA', 2976.1915475152973),
      ('LegPresentValues', [1243778.5097299085, 273427.0680831617]),
      ('ProductValue', 1517205.57781307),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 6548.875268071165),
      ('CashToSettleToday', 0.0)])

In [27]:
(179000-33000)*0.4

58400.0

# У большого поменяли премию на большой irs Package booked (Package number: "44363761").

In [31]:
#r=ps.get('19ca44cac520b7734029955501f9569d290a3bf9')

           
#Greeks new deal
r=ps_utils.pricing.XVA(ps.get('6b8d8a21682b698eab1d024259ae61a019bb886a')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
#r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])



#r.Product.CounterpartyCreditRating=8


r.price(algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-07-03 14:07:39,149 - Compute:c070d0b0091be2dbd0f4b84c955da737cbc8c225->120a1c29d26793e2717192c0506aa780540a8a61


Dict([('BCVA', -177783.58931487796),
      ('LegBCVA', [-144447.13275279236, -33362.0748741399]),
      ('Legs',
       [e449889afff64ca78a03b14f41e7da97622123c8,
        dd69b9fef6e3f48043b71f4fc0ada732258b5e13]),
      ('CVA', -180751.16615323228),
      ('DVA', 2967.576838354323),
      ('LegPresentValues', [1268820.0871339268, 267963.93250861706]),
      ('ProductValue', 1536784.0196425437),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 6494.118373861815),
      ('CashToSettleToday', 0.0)])

In [35]:
#r=ps.get('19ca44cac520b7734029955501f9569d290a3bf9')

           
#Greeks new deal
r=ps_utils.pricing.XVA(ps.get('264e7d64a9527488e72657fbf573f8fddb887f7e')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
#r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])

r.Product.NettingSets[0].MarginSets[0].Products[0]='e449889afff64ca78a03b14f41e7da97622123c8'


r.price(algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')


[INFO] 2023-07-03 14:11:32,340 - Compute:5429bf680a8980d196a5071d2accb23cbaebb116->169dc850429a97c892f5d750d9e4a42b75390acc


Dict([('BCVA', -182078.68845865212),
      ('LegBCVA', [-147880.45975923733, -34223.02625713992]),
      ('Legs',
       [e449889afff64ca78a03b14f41e7da97622123c8,
        6e8cf02cf569261d645f2b8f93c9a0649b7ca9c1]),
      ('CVA', -184957.30045714986),
      ('DVA', 2878.6119984977486),
      ('LegPresentValues', [1290334.5614794216, 273427.0680831617]),
      ('ProductValue', 1563761.629562583),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 6545.9228365506),
      ('CashToSettleToday', 0.0)])

In [36]:
147880*1.02

150837.6

In [37]:
3000*0.4

1200.0

# KAMER накоп 17 рейтинг

In [28]:
#r=ps.get('19ca44cac520b7734029955501f9569d290a3bf9')

           
#Greeks new deal
r=ps_utils.pricing.XVA(ps.get('099cd5316408bccbd63d0aed575f2b2bb5801746')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
#r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])



#r.Product.CounterpartyCreditRating=8


r.price(algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-07-03 13:45:09,975 - Compute:0982b9e89d903f30a034e16e9c613380dd5d0c57->3709753a1656c432ba2669599e9f5fc6faff8176


Dict([('BCVA', -1384.4704480055373),
      ('CVA', -1384.470449290811),
      ('DVA', 1.2852739166709276e-06),
      ('ProductValue', 152599.91417087198),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 9.45809594104177),
      ('CashToSettleToday', 0.0)])

# # MERKT030723  15 рейтинг Package booked (Package number: "44367201").

In [53]:
# new deal 
           
#Greeks new deal
r=ps_utils.pricing.XVA(ps.get('690952459a5cb49e9b817e01efcdc5354bf48d29')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
#r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])



#r.Product.CounterpartyCreditRating=8

r.Model.TypeName=skew_model

r.price(algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-07-03 16:42:50,467 - Compute:e3b85ab497e206eb9dcdef2f99d73acada74ea52->e04b3e66949b7777fa33147c043cb9efeefa8b51


Dict([('BCVA', -17170.23420068358),
      ('LegBCVA', [-11999.459660073095, -5170.884832115276]),
      ('Legs',
       [81de4f9958956286eecb0ee8aaaaed3b8d4485a2,
        3e5186a90277ce8a65476e82ee0d50a8d854a7ca]),
      ('CVA', -17699.343305860126),
      ('DVA', 529.1091051765462),
      ('LegPresentValues', [422053.1933043177, 227138.64509604362]),
      ('ProductValue', 649191.8384003611),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 1271.970477336016),
      ('CashToSettleToday', 0.0)])

In [50]:
# было
           
#Greeks new deal
r=ps_utils.pricing.XVA(ps.get('0c2f728a4db41ad351f168e0857d39899121c441')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
#r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])



#r.Product.CounterpartyCreditRating=8

r.Model.TypeName=skew_model

r.price(algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-07-03 16:41:46,730 - Compute:e99e92d9b45b2b5091a67c855004f72a79c7f6b3->0d0a4b6d79176f0b60ec30e9ad2fabdf05438cfd


Dict([('BCVA', -1409.3651249963852),
      ('LegBCVA', [-1409.3651249963852]),
      ('Legs', [b0499140312f607bc134547d1b64d2f4ff655860]),
      ('CVA', -1409.404585514956),
      ('DVA', 0.03946051857054275),
      ('LegPresentValues', [251282.10521523442]),
      ('ProductValue', 251282.10521523442),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 39.85611970024839),
      ('CashToSettleToday', 0.0)])

In [51]:
# стало
           
#Greeks new deal
r=ps_utils.pricing.XVA(ps.get('848499049518b6d6a11ff99d673caab23537cfb0')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
#r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])



#r.Product.CounterpartyCreditRating=8

r.Model.TypeName=skew_model

r.price(algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-07-03 16:41:49,924 - Compute:889c0451768a2d2edf0da5c54ab052ba84e852cf->ee99b8c20c65042218840d1eea96301358f7a26b


Dict([('BCVA', -18953.091928893697),
      ('LegBCVA',
       [-1411.0924940169982, -12260.506865669555, -5281.852754492862]),
      ('Legs',
       [b0499140312f607bc134547d1b64d2f4ff655860,
        81de4f9958956286eecb0ee8aaaaed3b8d4485a2,
        3e5186a90277ce8a65476e82ee0d50a8d854a7ca]),
      ('CVA', -19420.92177321082),
      ('DVA', 467.8298443171242),
      ('LegPresentValues',
       [251348.3679310361, 422274.5836650333, 227241.44001399356]),
      ('ProductValue', 900864.3916100627),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 1286.0099145463557),
      ('CashToSettleToday', 0.0)])

In [47]:
(18953-1409)*1.01

17719.44

In [48]:
#44367202
(-11995279+56843836-5563996)/r.Model.MarketDataSet.Spots['USDRUB']

441269.41211159626

In [52]:
#44367203
(-4998727+28343836-2384121)/r.Model.MarketDataSet.Spots['USDRUB']

235447.27538226085

In [57]:
(441269/422053*12000 + 235447/227138*5170)*1.01 +500

18584.53799899105

In [60]:
441269/422053*12000*1.01 +500

13171.821501091095

In [61]:
# new deal 
           
#Greeks new deal
r=ps_utils.pricing.XVA(ps.get('690952459a5cb49e9b817e01efcdc5354bf48d29')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
#r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])



r.Product.CounterpartyCreditRating=8

r.Model.TypeName=skew_model

r.price(algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-07-03 17:00:52,497 - Compute:9bc029ee4fa2a06c99ff2f2e37b5645b5d6f9a08->ab9c4da20b7bee366f371f1777e5410e71c8da82


Dict([('BCVA', -6406.124754185446),
      ('LegBCVA', [-4478.955043955151, -1927.1697102302955]),
      ('Legs',
       [81de4f9958956286eecb0ee8aaaaed3b8d4485a2,
        3e5186a90277ce8a65476e82ee0d50a8d854a7ca]),
      ('CVA', -6935.2338593619925),
      ('DVA', 529.1091051765462),
      ('LegPresentValues', [422053.1933043177, 227138.64509604362]),
      ('ProductValue', 649191.8384003611),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 1271.970477336016),
      ('CashToSettleToday', 0.0)])

In [62]:
441269/422053*4478*1.01

4728.70139015716

In [63]:
235447/227138*1927

1997.4921369387773

In [64]:
(13.2-4.8)

8.399999999999999

# RAZYU termination 18 rating

In [41]:
#r=ps.get('19ca44cac520b7734029955501f9569d290a3bf9')

           
#Greeks new deal
r=ps_utils.pricing.XVA(ps.get('89d7e8f2d999c09f353c5b81b3575b759beb5f64')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
#r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])

r.Model.TypeName=skew_model

#r.Product.CounterpartyCreditRating=8


r.price(algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-07-03 15:19:59,737 - Compute:d96e8df43d0ae0c74030ec62393e51565697b6c6->6d5357a9daccd2ce6df77e020218c2a6e372699b


Dict([('BCVA', -61114.654607202465),
      ('CVA', -147860.79729829312),
      ('DVA', 86746.14269109066),
      ('ProductValue', -2845221.2012836915),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 22240.681327429484),
      ('CashToSettleToday', 0.0)])